In [6]:
!pip install amrlib

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 1.3 MB/s eta 0:00:00m eta 0:00:010:02:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 4.2 MB/s eta 0:00:00m eta 0:00:010:00:06
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00m eta 0:00:010:00:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 4.8 MB/s eta 0:00:000:00:010:00:0102
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 4.8 MB/s eta 0:00:004.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 3.6 MB/s eta 0:00:00? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 4.7 MB/s eta 0:00:00m eta 0:00:010:00:04
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00m eta 0:00:010:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 4.7 MB/s eta 0:00:000:00:

In [16]:
import amrlib
stog = amrlib.load_stog_model()
graphs = stog.parse_sents(['This is a test of the system.', 'This is a second sentence.'])
for graph in graphs:
    print(graph)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

# ::snt This is a test of the system.
(t / test-01
      :ARG1 (s / system)
      :domain (t2 / this))
# ::snt This is a second sentence.
(s / sentence
      :ord (o / ordinal-entity
            :value 2)
      :domain (t / this))


In [20]:
!pip install word2number

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5568 sha256=846c2f845ed401773a2ccf08310ae5def0dfd23a70fd11b227d553d34139bb75
  Stored in directory: /home/masih/.cache/pip/wheels/5b/79/fb/d25928e599c7e11fe4e00d32048cd74933f34a74c633d2aea6
Successfully built word2number


In [22]:
import amrlib
gtos = amrlib.load_gtos_model()
sents, _ = gtos.generate(graphs)
for sent in sents:
    print(sent)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

This is system testing.
This is the second sentence.


In [1]:
import amrlib
import torch
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.utils.data import Dataset, DataLoader


with open('/home/masih/Desktop/data.txt', 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()

original_texts = lines[::2]
summaries = lines[1::2]

# Convert Text to AMR Graphs
stog = amrlib.load_stog_model()
graphs = stog.parse_sents(original_texts)

# Tokenize the AMR Graphs
# Convert graphs to strings and then to tokens
graph_strings = [str(graph) for graph in graphs]
graph_tokens = [graph_string.split() for graph_string in graph_strings]

# Apply Embeddings
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

max_seq_len = 10  
def get_bert_embeddings(tokens, max_seq_len):
    if len(tokens) > max_seq_len:
        tokens = tokens[:max_seq_len]
    elif len(tokens) < max_seq_len:
        tokens = tokens + ['[PAD]'] * (max_seq_len - len(tokens))
    inputs = tokenizer(tokens, padding='max_length', truncation=True, return_tensors='pt', max_length=max_seq_len)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    embeddings = outputs.last_hidden_state  
    return embeddings.squeeze(0)  

# Embeddings for each graph
graph_embeddings = [get_bert_embeddings(tokens, max_seq_len) for tokens in graph_tokens]

class EncoderDecoder(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_size):
        super(EncoderDecoder, self).__init__()
        self.encoder = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.decoder = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x, y):
        _, (hidden, _) = self.encoder(x)
        output, _ = self.decoder(y, hidden)
        output = self.fc(output)
        return output

embedding_dim = 768  
hidden_dim = 256
output_size = len(tokenizer.vocab)  

model = EncoderDecoder(embedding_dim, hidden_dim, output_size)

# Train the Model
class SummaryDataset(Dataset):
    def __init__(self, graph_embeddings, summaries, tokenizer, max_seq_len):
        self.graph_embeddings = graph_embeddings
        self.summaries = summaries
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len
    
    def __len__(self):
        return len(self.summaries)
    
    def __getitem__(self, idx):
        x = self.graph_embeddings[idx]
        summary = self.summaries[idx]
        summary_enc = self.tokenizer(summary, padding='max_length', truncation=True, return_tensors='pt', max_length=self.max_seq_len)
        y = summary_enc['input_ids'].squeeze()
        return x, y

dataset = SummaryDataset(graph_embeddings, summaries, tokenizer, max_seq_len)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    for i, (x, y) in enumerate(dataloader):
        x = x.view(x.size(0), max_seq_len, embedding_dim)
        # Forward pass
        outputs = model(x, y[:,:-1])
        loss = criterion(outputs.view(-1, output_size), y[:,1:].view(-1))
        # Backward 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}')

OutOfMemoryError: CUDA out of memory. Tried to allocate 78.00 MiB. GPU 0 has a total capacity of 7.78 GiB of which 34.62 MiB is free. Including non-PyTorch memory, this process has 7.73 GiB memory in use. Of the allocated memory 7.45 GiB is allocated by PyTorch, and 121.26 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)